In [ ]:
# # **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
# stable = True # Set to True for latest pip version or False for main branch in GitHub
# !pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [ ]:
!pip install tsai

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Thesis_exp')
!ls

In [ ]:
from tsai.all import *
computer_setup()

In [ ]:
# export
from tsai.imports import *
# from tsai.utils import *
from tsai.utils import *
from torch.nn.init import normal_
from fastai.torch_core import Module
from fastai.layers import *
from torch.nn.utils import weight_norm, spectral_norm


def total_params(m):
    "Give the number of parameters of a module and if it's trainable or not"
    params = sum([p.numel() for p in m.parameters()])
    trains = [p.requires_grad for p in m.parameters()]
    return params, (False if len(trains)==0 else trains[0])

In [ ]:
import numpy as np
import pandas as pd

print(get_UCR_univariate_list())
a = get_UCR_univariate_list()

# FCN、ResNet、xresnet、rescnn、LSTM、InceptionTime、OmniScaleCNN以及mWDN


all_dataset = [
              #  'ACSF1', 'Adiac', 'AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 
              #  'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 
              #  'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'Computers', 'CricketX', 'CricketY', 
              #  'CricketZ', 'Crop', 'DiatomSizeReduction', 'DistalPhalanxOutlineAgeGroup', 
              #  'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame', 
              #  'DodgerLoopWeekend', 'Earthquakes', 'ECG200', 'ECG5000', 'ECGFiveDays', 'ElectricDevices', 
              #  'EOGHorizontalSignal', 'EOGVerticalSignal', 'EthanolLevel', 'FaceAll', 'FaceFour', 'FacesUCR', 
              #  'FiftyWords', 'Fish', 'FordA', 'FordB', 'FreezerRegularTrain', 'FreezerSmallTrain', 'Fungi', 
              #  'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 
              #  'GunPoint', 'GunPointAgeSpan', 'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'Ham', 
              #  'HandOutlines', 'Haptics', 'Herring', 'HouseTwenty', 'InlineSkate', 'InsectEPGRegularTrain', 
              #  'InsectEPGSmallTrain', 'InsectWingbeatSound', 'ItalyPowerDemand', 'LargeKitchenAppliances', 
              #  'Lightning2', 'Lightning7', 'Mallat', 'Meat', 'MedicalImages', 'MelbournePedestrian', 
              #  'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect', 'MiddlePhalanxTW', 
              #  'MixedShapesRegularTrain', 'MixedShapesSmallTrain', 'MoteStrain', 'NonInvasiveFetalECGThorax1', 
              #  'NonInvasiveFetalECGThorax2', 'OliveOil', 'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme', 
              #  'PickupGestureWiimoteZ', 'PigAirwayPressure', 'PigArtPressure', 'PigCVP', 'PLAID', 'Plane', 
              #  'PowerCons', 'ProximalPhalanxOutlineAgeGroup', 'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 
              #  'RefrigerationDevices', 'Rock', 'ScreenType', 'SemgHandGenderCh2', 'SemgHandMovementCh2', 
              #  'SemgHandSubjectCh2', 'ShakeGestureWiimoteZ', 'ShapeletSim', 'ShapesAll', 'SmallKitchenAppliances', 
               'SmoothSubspace', 
              #  'SonyAIBORobotSurface1', 'SonyAIBORobotSurface2', 'StarLightCurves', 'Strawberry', 
              #  'SwedishLeaf', 'Symbols', 'SyntheticControl', 'ToeSegmentation1', 'ToeSegmentation2', 'Trace', 
              #  'TwoLeadECG', 'TwoPatterns', 'UMD', 'UWaveGestureLibraryAll', 'UWaveGestureLibraryX', 
              #  'UWaveGestureLibraryY', 'UWaveGestureLibraryZ', 'Wafer', 'Wine', 'WordSynonyms', 'Worms', 
              #  'WormsTwoClass', 'Yoga'
               ]


Abnormal_dataset = ['AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 'DodgerLoopDay', 'DodgerLoopGame', 'DodgerLoopWeekend', 
             'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 'PickupGestureWiimoteZ', 'PLAID',
             'ShakeGestureWiimoteZ', 'MelbournePedestrian']


df = pd.DataFrame(columns=['Database', 'FCN', 'ResNet', 'xresnet', 'ResCNN', 'LSTM', 'InceptionTime', 'mWDN'])



# for dsid in a[0:2]:
for dsid in all_dataset:
    
    # dsid = 'NATOPS' 
    print(dsid)
    bs = 64
    X, y, splits = get_UCR_data(dsid, return_split=False)
    print(X.shape)
    tfms  = [None, [Categorize()]]
    dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
    if dsid in Abnormal_dataset:
        batch_transforms = [TSStandardize(by_sample=True), Nan2Value()]
        dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2], batch_tfms=batch_transforms)
    else:
        dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])
    
    # FCN、ResNet、xresnet、rescnn、LSTM、InceptionTime、OmniScaleCNN以及mWDN
    archs = [(FCN, {}), (ResNet, {}), (xresnet1d34, {}), (ResCNN, {}), (LSTM, {'n_layers':1, 'bidirectional': True}), (InceptionTime, {}), (mWDN, {'levels': 4})]
    # archs = [(OmniScaleCNN, {})]

    results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
    ACC = []
    TIME = []
    for i, (arch, k) in enumerate(archs):
        model = create_model(arch, dls=dls, **k)
        print(model.__class__.__name__)
        learn = Learner(dls, model,  metrics=accuracy)
        start = time.time()
        # learn.fit_one_cycle(10, 1e-3)
        learn.fit_one_cycle(100, 1e-3)
        elapsed = time.time() - start
        vals = learn.recorder.values[-1]
        results.loc[i] = [arch.__name__, k, total_params(model)[0], vals[0], vals[1], vals[2], int(elapsed)]
        results.sort_values(by='accuracy', ascending=False, ignore_index=True, inplace=True)
        # clear_output()
        display(results)
        ACC.append(vals[2])
        TIME.append(int(elapsed))
    
    data = {
        'Database': dsid, 
        'FCN': ACC[0], 
        'ResNet': ACC[1],
        'xresnet':ACC[2],
        'ResCNN': ACC[3],
        'LSTM': ACC[4],
        'InceptionTime': ACC[5],
        'mWDN': ACC[6]
        }
    df = df.append(data, ignore_index=True)

    print(df)
    df.to_csv(f'./std_output_all_dl3.csv',encoding='gbk')
    


In [ ]:
# archs = [(XCM, {}),(MiniRocket, {})]
import numpy as np
import pandas as pd

print(get_UCR_univariate_list())
a = get_UCR_univariate_list()


all_dataset = [
               'ACSF1', 'Adiac', 'AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 
               'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 
               'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'Computers', 'CricketX', 'CricketY', 
               'CricketZ', 'Crop', 'DiatomSizeReduction', 'DistalPhalanxOutlineAgeGroup', 
               'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame', 
               'DodgerLoopWeekend', 'Earthquakes', 'ECG200', 'ECG5000', 'ECGFiveDays', 'ElectricDevices', 
               'EOGHorizontalSignal', 'EOGVerticalSignal', 'EthanolLevel', 'FaceAll', 'FaceFour', 'FacesUCR', 
               'FiftyWords', 'Fish', 'FordA', 'FordB', 'FreezerRegularTrain', 'FreezerSmallTrain', 'Fungi', 
               'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 
               'GunPoint', 'GunPointAgeSpan', 'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'Ham', 
               'HandOutlines', 'Haptics', 'Herring', 'HouseTwenty', 'InlineSkate', 'InsectEPGRegularTrain', 
               'InsectEPGSmallTrain', 'InsectWingbeatSound', 'ItalyPowerDemand', 'LargeKitchenAppliances', 
               'Lightning2', 'Lightning7', 'Mallat', 'Meat', 'MedicalImages', 'MelbournePedestrian', 
               'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect', 'MiddlePhalanxTW', 
               'MixedShapesRegularTrain', 'MixedShapesSmallTrain', 'MoteStrain', 'NonInvasiveFetalECGThorax1', 
               'NonInvasiveFetalECGThorax2', 'OliveOil', 'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme', 
               'PickupGestureWiimoteZ', 'PigAirwayPressure', 'PigArtPressure', 'PigCVP', 'PLAID', 'Plane', 
               'PowerCons', 'ProximalPhalanxOutlineAgeGroup', 'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 
               'RefrigerationDevices', 'Rock', 'ScreenType', 'SemgHandGenderCh2', 'SemgHandMovementCh2', 
               'SemgHandSubjectCh2', 'ShakeGestureWiimoteZ', 'ShapeletSim', 'ShapesAll', 'SmallKitchenAppliances', 
               'SmoothSubspace', 
               'SonyAIBORobotSurface1', 'SonyAIBORobotSurface2', 'StarLightCurves', 'Strawberry', 
               'SwedishLeaf', 'Symbols', 'SyntheticControl', 'ToeSegmentation1', 'ToeSegmentation2', 'Trace', 
               'TwoLeadECG', 'TwoPatterns', 'UMD', 'UWaveGestureLibraryAll', 'UWaveGestureLibraryX', 
               'UWaveGestureLibraryY', 'UWaveGestureLibraryZ', 'Wafer', 'Wine', 'WordSynonyms', 'Worms', 
               'WormsTwoClass', 'Yoga'
               ]


Abnormal_dataset = ['AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 'DodgerLoopDay', 'DodgerLoopGame', 'DodgerLoopWeekend', 
             'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 'PickupGestureWiimoteZ', 'PLAID',
             'ShakeGestureWiimoteZ', 'MelbournePedestrian']


df = pd.DataFrame(columns=['Database', 'XCM', 'MiniRocket'])



# for dsid in a[0:2]:
for dsid in all_dataset:
    
    # dsid = 'NATOPS' 
    print(dsid)
    bs = 64
    X, y, splits = get_UCR_data(dsid, return_split=False)
    print(X.shape)
    tfms  = [None, [Categorize()]]
    dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
    if dsid in Abnormal_dataset:
        batch_transforms = [TSStandardize(by_sample=True), Nan2Value()]
        dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2], batch_tfms=batch_transforms)
    else:
        dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])
    
    # FCN、ResNet、xresnet、rescnn、LSTM、InceptionTime、OmniScaleCNN以及mWDN
    archs = [(XCM, {}),(MiniRocket, {})]
    # archs = [(OmniScaleCNN, {})]

    results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
    ACC = []
    TIME = []
    for i, (arch, k) in enumerate(archs):
        model = create_model(arch, dls=dls, **k)
        print(model.__class__.__name__)
        learn = Learner(dls, model,  metrics=accuracy)
        start = time.time()
        # learn.fit_one_cycle(10, 1e-3)
        learn.fit_one_cycle(100, 1e-3)
        elapsed = time.time() - start
        vals = learn.recorder.values[-1]
        results.loc[i] = [arch.__name__, k, 0, vals[0], vals[1], vals[2], int(elapsed)]
        results.sort_values(by='accuracy', ascending=False, ignore_index=True, inplace=True)
        # clear_output()
        display(results)
        ACC.append(vals[2])
        TIME.append(int(elapsed))
    
    data = {
        'Database': dsid, 
        'XCM': ACC[0], 
        'MiniRocket': ACC[1],
        }
    df = df.append(data, ignore_index=True)

    print(df)
    df.to_csv(f'./std_output_all_newest_dl.csv',encoding='gbk')
    
